In [1]:
from pymongo import MongoClient, ASCENDING
from srs.database import connect_to_db
from srs.utilities import Sentence, tokenize
from nltk import pos_tag
from collections import Counter
import math
import word2vec
import os
import numpy as np
import random
import copy
import gzip
import ast
# Loading Word2Vec model
current_directory = os.path.dirname(os.path.realpath("__file__"))
model_path = os.path.join(current_directory[:-6], 'srs/predictor_data/text8.bin')
model = word2vec.load(model_path)
def sort_list(list, sort_index, reverse = True):
    list_sorted = sorted(list, key=lambda tup: tup[sort_index], reverse = reverse)
    return list_sorted

### Obtain the prod_dict:

In [2]:
def parse(path):
    g = gzip.open(path, 'r')
    for l in g:
        yield ast.literal_eval(l)


def construct_prod_dict(meta_file_path_list):
    """return a dictionary for product metadata"""
    prod_dict = {}
    for meta_file_path in meta_file_path_list:
        metaParser = parse(meta_file_path)
        client, db = connect_to_db()
        i = 0       
        print "Building the product dictionary for %s" % meta_file_path
        for meta in metaParser:
            i+=1
            if i % 100000 == 0:
                print i
            product_id = meta['asin']
            category = meta['categories'][0]
            product_name = ""
            brand = ""
            if 'title' in meta:
                inter = meta['title'].split()
                if len (inter) > 1:
                    product_name_short = inter[0] + ' ' + inter[1]
                else:
                    product_name_short = inter[0]
            if 'brand' in meta:
                brand = meta['brand']
            prod_dict[product_id]={'category': category, 'product_name': product_name_short, 'brand': brand}
        print i
    return prod_dict

In [6]:
Electronics_Meta_Path = '../../Datasets/Full_Reviews/meta_Electronics.json.gz'
Phone_Meta_Path = '../../Datasets/Full_Reviews/meta_Cell_Phones_and_Accessories.json.gz'

prod_dict = construct_prod_dict([Electronics_Meta_Path,Phone_Meta_Path])

Building the product dictionary for ../../Datasets/Full_Reviews/meta_Electronics.json.gz
100000
200000
300000
400000
498196
Building the product dictionary for ../../Datasets/Full_Reviews/meta_Cell_Phones_and_Accessories.json.gz
100000
200000
300000
346793


### The following functions accumulate all the sentences by category

In [14]:
def get_category_dict(prod_dict):
    """Build a dictionary whose key is the category tuple, and the value is a list of product_ids:"""
    client, db = connect_to_db()
    cursor = db.product_collection.find()
    category_dict = {}
    i = 0
    for product in cursor:
        i += 1   
        if i % 100000 == 0:
            print i
        category = product['category']
        category_short = tuple(category[:4]) #generally category is 4-tuple. Now limit to the first three tuple
        product_id = product['product_id']
        product_name = ""
        brand = ""
        if product_id in prod_dict:
            product_info = prod_dict[product_id]
            if 'product_name' in product_info:
                product_name = product_info['product_name']
            if 'brand' in product_info:
                brand = product_info['brand']

        if category_short not in category_dict:
            category_dict[category_short] = {"product_id": [product_id], "brand_list": [], "product_name_list": []}
        else:
            category_dict[category_short]['product_id'].append(product_id)
            
        if len(product_name) > 0:
            category_dict[category_short]['product_name_list'].append(product_name)
        if len(brand) > 0:
            if brand not in category_dict[category_short]['brand_list']:
                category_dict[category_short]['brand_list'].append(brand)
            
    client.close()
    print i
  
    return category_dict


def sort_category_dict(category_dict, isPrint = False):
    """Sort the categories according to the number of products in that category, and print them from top"""
    category_list_sorted = []
    category_list = []

    for key in category_dict:
        length = len(category_dict[key]['product_id'])
        category_list.append([key,length,key[:3],0])
    category_list_sorted = sorted(category_list, key=lambda tup: (tup[2],tup[1]), reverse=True)
    
    category_list_sorted_dict = {}
    for Id in range(len(category_list_sorted)):
        category_list_sorted[Id][3]=Id
        category = category_list_sorted[Id][0]
        category_dict[category]["category_id"] = Id
        category_list_sorted_dict[Id] = category_list_sorted[Id][:3]
    
    if isPrint:
        for Id in range(len(category_list_sorted)):
            print Id, category_list_sorted_dict[Id][:2]
        
    return category_list_sorted_dict


def combine_category_custom(category_dict_raw, category_list_sorted_dict):
    category_dict = copy.deepcopy(category_dict_raw)
    print "Number of categories in original set: %g"%len(category_dict_raw)
    print "Combined category ID:"
    f = open('Aspect_and_wordlist_txt/combined_dict.txt','r')
    for line in f:
        combine_info = eval(line)
        print combine_info
        if len(combine_info) > 0:
            Id_to_combine = combine_info[0]
            name_info = combine_info[1]
            category_name_combined = category_list_sorted_dict[name_info[0]][0][:name_info[1]]
            category_id = category_dict_raw[category_list_sorted_dict[name_info[0]][0]]["category_id"]
            new_prod_id_list = []
            new_product_name_list = []
            new_brand_list = []
            for Id in Id_to_combine:
                category_name = category_list_sorted_dict[Id][0]
                new_prod_id_list += category_dict[category_name]["product_id"]
                new_product_name_list += category_dict[category_name]["product_name_list"]
                new_brand_list += category_dict[category_name]["brand_list"]
                category_dict.pop(category_name, 0)
            category_dict[category_name_combined] = {"category_id": category_id,"product_id": new_prod_id_list,\
                        "product_name_list": new_product_name_list, "brand_list": new_brand_list}
    f.close()
    print "Number of categories in the new dict: %g"%len(category_dict)
      
    return category_dict


def combine_small_category(category_dict_raw, category_list_sorted, prod_num_threshold = 100, shrink_level = 3):
    category_dict = copy.deepcopy(category_dict_raw)
    i = 0
    for i in range(len(category_list_sorted)):
        i += 1
        category_name = category_list_sorted[-i][1]
        prod_num = category_list_sorted[-i][0]
        if prod_num > prod_num_threshold:
            break
        if len(category_name) > shrink_level:
            category_name_shrink = category_name[:shrink_level]
            if category_name_shrink in category_dict:
                category_dict[category_name_shrink] += category_dict[category_name]
                category_dict.pop(category_name,0)
                print "{0} combined into {1}".format(category_name_shrink, category_name)
            else:
                print "{0} not combined".format(category_name_shrink)
        else:
            print "{0} length not enough.".format(category_name)
    
    return category_dict


def save_category_dict_to_db(category_dict, dropPrevious = False):
    client, db = connect_to_db()
    db_category_data = db.category_data
    if dropPrevious == True:
        db_category_data.delete_many({})
    for category in category_dict:
        query = {"category_id": category_dict[category]["category_id"]}
        update_field = {"category": list(category),\
                        "prod_id_list": category_dict[category]["product_id"], \
                        "brand_list":  category_dict[category]["brand_list"],\
                        "product_name_list": category_dict[category]["product_name_list"]}
        db_category_data.update_one(query, {"$set": update_field}, True)
        
    client.close()


def show_category_dict_info(category_dict, min_prod_num = 1000):
    new_list = []
    for category in category_dict:
        new_list.append([len(category_dict[category]["product_id"]),category,category_dict[category]["category_id"]])
    
    new_list = sorted(new_list, key=lambda tup: tup[0], reverse=True)
    
    for item in new_list:
        if int(item[0]) < min_prod_num:        
            break
        print "{0},{1},{2}".format(item[0],item[1],item[2])


def get_sentence_from_category(category_list):
    """Obtain all the review sentences from a list of category tuple:"""
    if isinstance(category_list, dict):
        category_lists = [category_list]
    else:
        category_lists = category_list
    
    category_content_list = []
    
    for category in category_lists:
        print "{0}:".format(category)
        client, db = connect_to_db()
        product_id_list = category_dict[category]["product_id"]
        category_contents = {"category": category,"sentence_list": [], "brand_list": category_dict[category]["brand_list"],\
                            "product_name_list": category_dict[category]["product_name_list"]}
        review_num = 0
        for product_id in product_id_list:
            query_res = list(db.product_collection.find({"product_id": product_id}))
            contents = query_res[0]["contents"]
            category_contents['sentence_list'] += contents
            review_num += len(query_res[0]["review_ids"])
        print "  ({0}, {1}, {2})".format(len(product_id_list), review_num, len(category_contents['sentence_list']))      
        category_content_list.append(category_contents)
        
    client.close()

    return category_content_list


def get_sentence_from_category_ensemble(category_dict, max_prod_chosen = 500, min_product_level = 500):
    client, db = connect_to_db()
    full_sentence_list = []
    print "Getting product categories: (num_sentence_chosen, category):"
    for category in category_dict:
        if len(category_dict[category]) < min_product_level:
            continue
        product_id_list = category_dict[category]["product_id"]
        random.shuffle(product_id_list)
        new_sentence = []
        for product_id in product_id_list[:max_prod_chosen]:
            query_res = list(db.product_collection.find({"product_id": product_id}))
            contents = query_res[0]["contents"]
            new_sentence += contents
        print len(new_sentence),category
        full_sentence_list += new_sentence
    client.close()
    print "Number of sentences: {0}".format(len(full_sentence_list))
    
    all_category_content = {"sentence_list": full_sentence_list}
    return all_category_content

In [12]:
category_dict_raw = get_category_dict(prod_dict)

100000
200000
300000
400000
500000
600000
700000
793315


In [15]:
category_list_sorted_dict = sort_category_dict(category_dict_raw, isPrint = False)
category_dict = combine_category_custom(category_dict_raw, category_list_sorted_dict)
save_category_dict_to_db(category_dict, dropPrevious = False)

Number of categories in original set: 512
Combined category ID:
[[16, 17], [17, 2]]
[[20, 21, 22, 23, 26, 28, 29, 30, 31, 32], [22, 3]]
[[38, 39, 40, 41, 42], [38, 3]]
[[105, 106], [105, 3]]
[[108, 109, 110, 111, 112], [112, 3]]
[[139, 140, 141], [139, 3]]
[[176, 177, 178, 179, 180, 181, 182, 183, 184], [176, 3]]
[[277, 278, 279, 280], [277, 3]]
[[282, 283, 284, 285, 286, 287, 288], [282, 3]]
[[297, 298, 299, 300, 301], [297, 3]]
[[302, 303, 304], [302, 3]]
[[308, 309, 310, 311, 312, 313, 314], [308, 3]]
[[316, 321], [316, 3]]
[[322, 323, 324, 325, 326, 327, 328, 329, 330, 331], [322, 3]]
[[356, 357, 358, 359, 360], [356, 3]]
[[362, 363, 364, 365, 366], [366, 3]]
[[367, 368, 369, 370], [367, 3]]
[[371, 372, 373, 374, 375, 376, 377], [371, 3]]
[[379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393], [379, 3]]
[[399, 400], [399, 3]]
[[417, 418, 419, 420, 421], [417, 3]]
[[427, 428, 429, 430], [427, 2]]
[[433, 434, 435, 436, 437, 438], [433, 2]]
[[439, 440, 441, 442, 

In [22]:
show_category_dict_info(category_dict, min_prod_num = 1000)

207742,(u'Cell Phones & Accessories', u'Cases', u'Waterproof Cases'),439
25256,(u'Cell Phones & Accessories', u'Accessories', u'Accessory Kits'),474
25245,(u'Electronics', u'Computers & Accessories', u'Touch Screen Tablet Accessories', u'Cases & Sleeves'),129
21726,(u'Electronics', u'Portable Audio & Video', u'MP3 Players & Accessories', u'MP3 Player Accessories'),78
16275,(u'Electronics', u'Computers & Accessories', u'Laptop & Netbook Computer Accessories', u'Batteries'),155
15453,(u'Electronics', u'Camera & Photo', u'Accessories', u'Batteries & Chargers'),332
15195,(u'Cell Phones & Accessories', u'Accessories', u'Screen Protectors'),450
15051,(u'Electronics', u'Computers & Accessories', u'Laptop & Netbook Computer Accessories', u'Chargers & Adapters'),156
13548,(u'Electronics', u'Computers & Accessories', u'Laptop & Netbook Computer Accessories', u'Bags & Cases'),157
13115,(u'Electronics', u'Accessories & Supplies', u'Audio & Video Accessories', u'Cables & Interconnects'),401
11747,(

In [ ]:
# all_category_content = get_sentence_from_category_ensemble(category_dict, max_prod_chosen = 1000, min_product_level = 0)
# get_tf_idf(all_category_content, is_idf_db = False)

### Aspect extraction: the following functions collects sentences from one category, obtain each word's tf-idf score, and choose aspect candidates:

In [28]:
def get_category_word_scores(category_id_list):
    """Get tf-idf score for each word
       The dictionary records for each word as a key, the [num_word, num_doc] value, where num_word means the number of 
       that word in the sentence_list, and num_doc means the number of sentences this word appears in.
    """
    
    client, db = connect_to_db()
    db_word_score_list = db.word_score_list
    db_product_collection = db.product_collection
    db_category_data = db.category_data
    # Collecting each word's data for that category
    for category_id in category_id_list:
        query_category = list(db_category_data.find({"category_id": category_id}))
        if len(query_category) == 0:
            print "{0} not in db, skip.".format(category_id)
            continue
        category_content = query_category[0]
        category = category_content["category"]
        prod_id_list = category_content["prod_id_list"]
        prod_num = len(prod_id_list)
        if prod_num < 10:
            continue
        
        word_statistics = {}
        
        i = 0
        for product_id in prod_id_list:          
            query_res = list(db_product_collection.find({"product_id": product_id}))
            contents = query_res[0]["contents"]
            for sentence in contents:
                i += 1
                if i % 50000 == 0:
                    print i
                tokens = tokenize(sentence, stem = False)
                tokens_count = Counter(tokens)
                for word in tokens_count:        
                    if word not in word_statistics:
                        word_statistics[word] = [tokens_count[word], 1]
                    else:
                        word_statistics[word][0] += tokens_count[word]
                        word_statistics[word][1] += 1
        
        total_num_doc = i
        print "Id: {0}, num_prod: {1}, num_sentence: {2}".format(category_id, prod_num, total_num_doc)
        print "{0}".format(category)
        word_scores = []
        
        max_word_freq = 0
        for word in word_statistics:
            if word_statistics[word][0] > max_word_freq:
                max_word_freq = word_statistics[word][0]
        
        # Calculating tf-idf for the category
        for word in word_statistics:               
            word_rawdata = word_statistics[word]
            word_freq = word_rawdata[0]
            num_doc = word_rawdata[1]
            tf = float(word_freq) / max_word_freq 
                                   
            idf_category = math.log(float(total_num_doc)/(num_doc))
            query_idf = list(db_word_score_list.find({"word": word}))
            if len(query_idf) > 0:
                idf = query_idf[0]["full_word_score"][2]
            else:
                idf = idf_category
            word_scores.append([word, tf * idf, tf, idf_category, idf, word_freq, num_doc])
                                   
        word_statistics.clear()
        word_scores.sort(key=lambda tup: tup[1], reverse=True)
        
        # Obtaining brand_list words and product_name words:
        brand_list = category_content["brand_list"]
        product_name_list = category_content["product_name_list"]
        brand_word_list = []
        product_name_word_list =[]
        for brand in brand_list:     
            brand_word = brand.split()[0]
            if brand_word not in brand_word_list:
                brand_word_list.append(brand_word)
        for product_name in product_name_list:   
            product_name_word = product_name.split()[0]
            if product_name_word not in product_name_word_list:
                product_name_word_list.append(product_name_word)       
        
        # Update database:
        query = {"category_id": category_id}
        update_field = {"word_scores": word_scores, "brand_word_list": brand_word_list,\
                        "product_name_word_list": product_name_word_list}
        db_category_data.update_one(query, {"$set": update_field}, True)
        
        # Get aspect_cadidate:
        if prod_num < 20:
            num_candidate = 60
        else:
            num_candidate = 50
        get_aspect_cadidate([category_id], num_candidate, ["NN"], db_category_data, db_word_score_list)
  
    client.close()                          
    

def get_excluded_words():
    f = open("Aspect_and_wordlist_txt/excluded_words.txt",'r')
    excluded_words = eval(f.read())
    return excluded_words


def get_aspect_cadidate(category_id_list, num_candidate = 30, tag_list = ["NN"], db_category_data = None, db_word_score_list = None):
    '''Get cadidate aspects from word_tf_idf. Only words whose tag belong to tag_list and score > threshold will pass'''  
    # check if db cursor is given:
    external_db = True
    if (not db_category_data) or (not db_word_score_list):
        external_db = False
        client, db = connect_to_db()
        if not db_category_data:
            db_category_data = db.category_data
        if not db_word_score_list:
            db_word_score_list = db.word_score_list

    for category_id in category_id_list: 
        full_word_freq_thresh = 30
        full_num_doc_thresh = 10
        
        query_category = list(db_category_data.find({"category_id": category_id}))
        if len(query_category) == 0:
            print "{0} not in db, skip.".format(category_id)
            continue
        category_content = query_category[0]
        category = category_content["category"]
        if external_db == False:
            print category 
        if "word_scores" not in category_content:
            print "{0} don't have word_scores, skip.".format(category_id)
            continue
        word_scores = category_content["word_scores"]
        brand_word_list = category_content["brand_word_list"]
        product_name_word_list = category_content["product_name_word_list"]
        num_prod = len(category_content["prod_id_list"])
        if num_prod < 50:
            full_word_freq_thresh = int(num_prod/ 4)
            full_num_doc_thresh = int(num_prod / 8)
                    
        aspect_candidate = []
        j = 0
        excluded_words = get_excluded_words()
        for word_data in word_scores:
            word = word_data[0]
            # Various criterior to exclude the word:
            if len(word) == 1:
                continue
            if word_data[5] <= 1:
                continue
            if word in excluded_words:
                continue
            query_idf = list(db_word_score_list.find({"word": word}))
            if len(query_idf) > 0:
                full_word_score = query_idf[0]["full_word_score"]
                full_word_freq = full_word_score[4]
                full_num_doc = full_word_score[5]
                if full_word_freq < full_word_freq_thresh or full_num_doc < full_num_doc_thresh:
                    continue            
            if word in brand_word_list:
                if full_word_freq < 100:
                    continue
            if word in product_name_word_list:
                if full_word_freq < 200:
                    continue                        
                                  
            word_tag = pos_tag([word])[0][1]
            # If the tag is in tag_list:
            if word_tag in tag_list:
                j += 1
                aspect_candidate.append(word_data)
                print "%s     \t%0.2f\t%0.2f\t%0.2f\t%0.2f\t%g\t%g"%(word, word_data[1],word_data[2],word_data[3],word_data[4],\
                                                               word_data[5], word_data[6])
            if j > num_candidate:
                break
               
        # Update database:
        query = {"category_id": category_id}
        update_field = {"aspect_candidate": aspect_candidate}
        db_category_data.update_one(query, {"$set": update_field}, True)      
        print
    if external_db == False:
        client.close()



def save_word_score_to_db(category_content_list, isRewrite = False):
    client, db = connect_to_db()
    db_word_score_list = db.word_score_list
    if isRewrite == True:
        db_word_score_list.delete_many({})
        db_word_score_list.create_index([("word", ASCENDING)])
        db_word_score_list.create_index([("category", ASCENDING)])
    
    if isinstance(category_content_list, dict):
        category_content_lists = [category_content_list]
    else:
        category_content_lists = category_content_list
    
    for category_content in category_content_lists:
        word_tf_idf = category_content["word_tf_idf"]
        category = category_content["category"]
        i = 0
        for word_data in word_tf_idf:
            i += 1
            if i % 50000 == 0:
                print i
            word = word_data[0]
            word_score = word_data[1:]
            query = {"word": word}
            update_field = {"category": category, "word_score": word_score}

            db_word_score_list.update_one(query, {"$set": update_field}, True)       
        print "{0}: Total number of words: {1}".format(category, len(word_tf_idf))
    
    client.close()


def get_category_data_from_db(category_id, request_field_list):
    client, db = connect_to_db()
    db_category_data = db.category_data
    result = []
    if not isinstance(request_field_list, list):
        request_field_list = [request_field_list]
        isOnefield = True
    query = list(db_category_data.find({"category_id": category_id}))
    if len(query) > 0:
        query = query[0]
        for request_field in request_field_list:        
            if request_field in query:
                result.append(query[request_field])
            else:
                print "{0} not in category {1}".format(request_field, category_id)
                result.append([])
    else:
        print "category {0} not in db".format(category_id)
        result = [[] for request_field in request_field_list]
    if isOnefield == True:
        result = result[0]
    return result

### Wordlist generation:

In [41]:
def get_similarity(word1, word2):
    """Find the similarity between two words, which equals the dot product of their vectors"""
    similarity = 0
    word1=word1.lower()
    word2=word2.lower()
    if word1 in model and word2 in model:
        word1_vec = model[word1]
        word2_vec = model[word2]
        similarity = np.dot(word1_vec, word2_vec)
    return similarity

def get_wordlist_from_aspect_candidates(seed_word, word_tf_idf, similarity_threshold, score_threshold):
    """Method 1: directly find the word list from all words whose similarity with the seed_word and tf-idf score are above 
    certain threshold"""
    word_list = []
    for word_data in word_tf_idf:
        word = word_data[0]
        tf_idf = word_data[1]
        if tf_idf > score_threshold:
            similarity = get_similarity(seed_word, word)
            if similarity > similarity_threshold:
                word_list.append([word, similarity, tf_idf])              
    word_list_sorted = sorted(word_list, key=lambda tup: tup[1], reverse=True)
    return word_list_sorted


def get_wordlist_by_tf_idf(seed_word_list, category_id):
    """Method 2: Find the word_list who can distinguish the chosen sentences from other sentences"""
    
    # For each seed_word in seed_word_list, get the word_data for all sentences. word_data has 4 fields [num_word_total, num_doc_total, 
    # num_word_in_topic, num_doc_in_topic, similarity with seed_word], the first two are from all sentences, and the latter two are from the sentences 
    # that contain the seed_word.   
    client, db = connect_to_db()
    db_category_data = db.category_data
    db_product_collection = db.product_collection
    db_category_collection = db.category_collection
    
    category_content = list(db_category_data.find({"category_id": category_id}))[0]
    num_seed_word = len(seed_word_list)
    category = category_content["category"]
    word_statistics_dic_list = [{} for i in range(num_seed_word)]
    
    num_sentence_topic_list = [0 for k in range(num_seed_word)]
    num_sentence_total_list = [0 for k in range(num_seed_word)]
    i = 0
    print "{0}".format(category)
    print "Number of sentences processed:"
    
    prod_id_list = category_content["prod_id_list"]
    prod_num = len(prod_id_list)
        
    for product_id in prod_id_list:          
        query_res = list(db_product_collection.find({"product_id": product_id}))
        contents = query_res[0]["contents"]
        for sentence in contents:
            i += 1
            if i % 50000 == 0:
                print i
            tokens = tokenize(sentence, stem = False)
            tokens_count = Counter(tokens)
            for word in tokens_count:
                # check for each seed_word:
                for k in range(num_seed_word):
                    seed_word = seed_word_list[k]
                    if seed_word in sentence: 
                        num_sentence_topic_list[k] += 1
                        num_sentence_total_list[k] += 1
                        if word not in word_statistics_dic_list[k]:
                            word_statistics_dic_list[k][word] = [tokens_count[word], 1, tokens_count[word], 1]
                        else:
                            word_statistics_dic_list[k][word][0] += tokens_count[word]
                            word_statistics_dic_list[k][word][1] += 1
                            word_statistics_dic_list[k][word][2] += tokens_count[word]
                            word_statistics_dic_list[k][word][3] += 1
                    else:
                        num_sentence_total_list[k] += 1
                        if word not in word_statistics_dic_list[k]:
                            word_statistics_dic_list[k][word] = [tokens_count[word], 1, 0, 0]
                        else:
                            word_statistics_dic_list[k][word][0] += tokens_count[word]
                            word_statistics_dic_list[k][word][1] += 1

    # Get the maximum word frequency for each seed_word group:
    word_tf_idf_ratio_list = [[] for k in range(num_seed_word)]
    max_num_word_total_list =[0 for k in range(num_seed_word)]
    max_num_word_topic_list =[0 for k in range(num_seed_word)] 
    for k in range(num_seed_word):    
        for word in word_statistics_dic_list[k]:
            word_data = word_statistics_dic_list[k][word]
            if word_data[0] > max_num_word_total_list[k]:
                max_num_word_total_list[k] = word_data[0]
            if word_data[2] > max_num_word_topic_list[k]:
                max_num_word_topic_list[k] = word_data[2]
    
    # Get tf_idf adjusted ratio for each word, to measure how this word can distinguish the topic sentences:
    for k in range(num_seed_word):        
        for word in word_statistics_dic_list[k]:
            word_data = word_statistics_dic_list[k][word]
            num_word_total = word_data[0]
            num_word_topic = word_data[2]
            num_doc_total = word_data[1]   
            num_doc_topic = word_data[3]
        
            if num_doc_topic == 0 or num_doc_total == 0:
                word_tf_idf_ratio_list[k].append([word, 0, 0, 0])
                continue

            tf_topic = float(num_word_topic) / max_num_word_topic_list[k]
            tf_total = float(num_word_total) / max_num_word_total_list[k]
            tf_ratio = (tf_topic/num_word_topic) / (tf_total/num_doc_total)

            idf_topic = math.log(float(num_sentence_topic_list[k]) / num_doc_topic)
            idf_total = math.log(float(num_sentence_total_list[k]) / num_doc_total) 

            word_tf_idf_ratio_list[k].append([word, 0, 0, tf_topic * math.log(tf_ratio) * idf_total ** 2, tf_topic, tf_total, tf_ratio, idf_topic, idf_total])

        word_tf_idf_ratio_list[k].sort(key=lambda tup: tup[3], reverse=True)
 
    word_tf_idf_ratio_list = [item[:300] for item in word_tf_idf_ratio_list]
    query = {"category_id": category_id}
    update_field = {"word_tf_idf_ratio_list": word_tf_idf_ratio_list,"seed_word_list": seed_word_list}
    db.category_data.update_one(query, {"$set": update_field}, True)
    
    client.close()
    return word_tf_idf_ratio_list


def generate_wordlist_dict(seed_word_list, word_tf_idf_ratio_list, num_words_in_list, sim_slope = 1, sim_intercept = 0.2, isPrint = True):
    num_seed_word = len(seed_word_list)
    for k in range(num_seed_word): 
        length = len(word_tf_idf_ratio_list[k])
        for j in range(min(range(200),length)):
            word_data = word_tf_idf_ratio_list[k][j]
            word = word_data[0]
            similarity = get_similarity(word, seed_word_list[k])
            sim_amplify = sim_intercept + similarity * sim_slope
            word_tf_idf_ratio_list[k][j][2] = sim_amplify
            argument = 1 + word_tf_idf_ratio_list[k][j][3] * sim_amplify
            if argument > 0:
                word_tf_idf_ratio_list[k][j][1] = math.log(argument) 
            else:
                word_tf_idf_ratio_list[k][j][1] = -1
        word_tf_idf_ratio_list[k].sort(key=lambda tup: tup[1], reverse=True)

    word_tf_idf_ratio_list = [item[:num_words_in_list] for item in word_tf_idf_ratio_list]
    wordlist_dict = {}
    for k in range(num_seed_word):
        wordlist_dict[seed_word_list[k]] = [[word_data[0],word_data[1]] for word_data in word_tf_idf_ratio_list[k][:num_words_in_list]] 
    
    if isPrint == True:
        for aspect in wordlist_dict:
            print '"{0}": '.format(aspect)
            print '  ',
            for word_data in wordlist_dict[aspect]:
                print '"%s", %0.2f;'%(word_data[0], word_data[1]),
            print
    
    return wordlist_dict

def prune_wordlist_dict(wordlist_dict, excluded_word_external = [], preserve_top = False, isPrint = True):
    word_location = {}
    excluded_wordlist = ["good", "great", "well", "bad", "worse","better", "camera"] + excluded_word_external
    # Obtaining each word's aspects and score in that aspect:   
    if preserve_top == True:
        for aspect in wordlist_dict:
            for word_data in wordlist_dict[aspect]:
                word = word_data[0]
                score = word_data[1]
                if word in word_location:
                    word_location[word].append([aspect, score])
                else:
                    word_location[word]=[[aspect, score]]             

        # Sort each word's aspect, and only keep the word in highest score aspect:   
        wordlist_dict_pruned = {}
        for word in word_location:
            if word in excluded_wordlist:
                continue
            aspect_sorted = sort_list(word_location[word], 1)
            aspect_chosen = aspect_sorted[0] # Choose the first one
            aspect = aspect_chosen[0]
            if aspect in wordlist_dict_pruned:
                wordlist_dict_pruned[aspect].append([word, aspect_chosen[1]])
            else:
                wordlist_dict_pruned[aspect] = [[word, aspect_chosen[1]]]
    else:
        wordlist_dict_pruned = {}
        for aspect in wordlist_dict:
            wordlist = []
            for word in wordlist_dict[aspect]:
                if word[0] not in excluded_wordlist:
                    wordlist.append(word) 
            wordlist_dict_pruned[aspect] = wordlist
    
    # Sort each word
    for word in wordlist_dict_pruned:
        wordlist = wordlist_dict_pruned[word]
        wordlist_dict_pruned[word] = sort_list(wordlist, 1)
    
    if isPrint == True:
        for aspect in wordlist_dict_pruned:
            print '"{0}": '.format(aspect)
            print '  ',
            for word_data in wordlist_dict_pruned[aspect]:
                print '"%s", %0.2f;'%(word_data[0], word_data[1]),
            print
        
    return wordlist_dict_pruned

def iterate_wordlist_dict(wordlist_dict,sentence_list, num_words_in_list, sim_slope = 0.5, sim_intercept = 0.2):
    num_aspect = len(wordlist_dict)
    word_statistics_dic_list = [{} for i in range(num_aspect)]
    num_sentence_topic_list = [0 for k in range(num_aspect)]
    num_sentence_total_list = [0 for k in range(num_aspect)]
    
    i = 0
    for sentence in sentence_list: 
        i += 1
        if i % 50000 == 0:
            print i
        tokens = tokenize(sentence, stem = False)
        tokens_count = Counter(tokens)


def writeWordlistDictToDB(category_id, wordlist_dict):
    # Update database:    
    client, db = connect_to_db()
    query = {"category_id": category_id}
    update_field = {"wordlist_dict": wordlist_dict}
    db.category_data.update_one(query, {"$set": update_field}, True)
    category = list(db.category_data.find({"category_id": category_id}))[0]["category"]
    
    query_category = {"category_id": category_id}
    update_field_category = {"category": category, "wordlist_dict": wordlist_dict}
    db.category_collection.update_one(query_category, {"$set": update_field_category}, True)
    client.close()


def getWordlistDictFromDB(category):
    client, db = connect_to_db()
    category_collection = db.category_collection
    query_res = list(category_collection.find({"category": category}))
    disconnect_db(client)

    if len(query_res) < 1:
        raise Exception('Category: {0} not found in database'.format(category))
    elif len(query_res) > 1:
        raise Exception('Category: {0} found multiple occurances in database'.format(category))

    result = query_res[0]
    wordlistDictWithWeights = result['wordlist_dict']
    wordlistDict = {}
    for aspect in wordlistDictWithWeights:
        wordlistDict[aspect] = [sublist[0] for sublist in wordlistDictWithWeights[aspect]]

    return wordlistDict

In [32]:
category_id = 308 # Camera
seed_word_list = ["batteries", "pictures","price","zoom","ease of use", "detection","design","video","quality","screen","size"]
sim_slope = 1
sim_intercept = 0.2

# word_tf_idf_ratio_list = get_wordlist_by_tf_idf(seed_word_list, category_id)
wordlist_dict = generate_wordlist_dict(seed_word_list, word_tf_idf_ratio_list, 10, sim_slope, sim_intercept)
wordlist_dict_pruned = prune_wordlist_dict(wordlist_dict)
writeWordlistDictToDB(category_id, wordlist_dict)

"pictures": 
   "pictures", 3.54; "camera", 1.28; "takes", 1.05; "great", 1.00; "good", 0.94; "take", 0.93; "quality", 0.90; "flash", 0.84; "picture", 0.80; "videos", 0.75;
"price": 
   "price", 4.55; "prices", 2.11; "quality", 1.95; "priced", 1.92; "good", 1.87; "range", 1.86; "camera", 1.85; "better", 1.56; "reasonable", 1.53; "worth", 1.51;
"zoom": 
   "zoom", 4.30; "optical", 2.43; "x", 2.34; "zooming", 2.01; "camera", 1.92; "lens", 1.91; "video", 1.80; "zoomed", 1.70; "digital", 1.69; "mm", 1.52;
"ease of use": 
   "easy", 2.68; "use", 1.89; "camera", 1.04; "pictures", 1.00; "great", 0.98; "takes", 0.88; "good", 0.72; "carry", 0.70; "operate", 0.68; "small", 0.68;
"detection": 
   "detection", 6.34; "face", 4.12; "phase", 3.44; "blink", 3.31; "smile", 3.29; "mode", 3.20; "stabilization", 2.98; "focus", 2.79; "optical", 2.72; "auto", 2.71;
"batteries": 
   "battery", 4.20; "batteries", 2.25; "charge", 1.86; "extra", 1.81; "camera", 1.75; "charger", 1.74; "rechargeable", 1.62; "comp

In [44]:
wordlist_dict_pruned = prune_wordlist_dict(wordlist_dict)
writeWordlistDictToDB(308,wordlist_dict_pruned)

"pictures": 
   "pictures", 3.54; "takes", 1.05; "take", 0.93; "quality", 0.90; "flash", 0.84; "picture", 0.80; "videos", 0.75;
"price": 
   "price", 4.55; "prices", 2.11; "quality", 1.95; "priced", 1.92; "range", 1.86; "reasonable", 1.53; "worth", 1.51;
"zoom": 
   "zoom", 4.30; "optical", 2.43; "x", 2.34; "zooming", 2.01; "lens", 1.91; "video", 1.80; "zoomed", 1.70; "digital", 1.69; "mm", 1.52;
"ease of use": 
   "easy", 2.68; "use", 1.89; "pictures", 1.00; "takes", 0.88; "carry", 0.70; "operate", 0.68; "small", 0.68;
"detection": 
   "detection", 6.34; "face", 4.12; "phase", 3.44; "blink", 3.31; "smile", 3.29; "mode", 3.20; "stabilization", 2.98; "focus", 2.79; "optical", 2.72; "auto", 2.71;
"batteries": 
   "battery", 4.20; "batteries", 2.25; "charge", 1.86; "extra", 1.81; "charger", 1.74; "rechargeable", 1.62; "compartment", 1.39; "lithium", 1.30; "use", 1.29;
"design": 
   "design", 5.79; "designed", 4.74; "designs", 2.89; "flaw", 2.89; "designers", 2.55; "lens", 2.50; "quality",

In [46]:
category = (u'Electronics', u'Portable Audio & Video', u'Portable DVD Players') #(222, 3977, 22476)
category0 = ("Electronics", "Camera & Photo", "Digital Cameras") #(7917, 203926, 1726478)
category1 = (u'Cell Phones & Accessories', u'Cell Phones') # (743, 32200, 221681)
category2 = (u'Electronics', u'Accessories & Supplies', u'Batteries, Chargers & Accessories') #(7472, 53089, 206542)
category3 = (u'Electronics', u'Computers & Accessories', u'Monitors') #(3731, 51922, 364001)
category4 = (u'Electronics', u'Camera & Photo', u'Lenses') #(3737, 61874, 453186)
category5 = (u'Electronics', u'Computers & Accessories', u'Tablets') #(2439, 99311, 781032)
category6 = (u'Electronics', u'Television & Video', u'Televisions') #(4227, 141665, 1291788)
category7 = (u'Cell Phones & Accessories', u'Accessories', u'Headsets')
category8 = (u'Electronics', u'Camera & Photo', u'Bags & Cases')
category9 = (u'Cell Phones & Accessories', u'Accessories', u'Screen Protectors')
category10 = (u'Electronics', u'Portable Audio & Video', u'MP3 Players & Accessories')
category11 = (u'Electronics', u'Camera & Photo', u'Accessories')
category_list =[category, category1, category2, category3, category4, category5, category6, category7, category8, category9, \
                category10, category11]
category_id_list = []
for category in category_list:
    category_id = category_dict[category]['category_id']
    category_id_list.append(category_id)
category_id_list

[73, 433, 399, 153, 297, 136, 22, 456, 322, 450, 80, 340]